In [1]:
# ! pip install wandb # colab only

import math
from collections import defaultdict, OrderedDict
import numpy as np
import torch
from torch import nn
from torchinfo import summary
from pprint import pprint
import wandb
import config as cfg

def download_and_unzip(url, save_dir='.'):
  # downloads and unzips url, if not already downloaded
  import os
  from urllib.request import urlopen
  from io import BytesIO
  from zipfile import ZipFile
  fname = url.split('/')[-1][:-4] if save_dir == '.' else save_dir
  if fname not in os.listdir():
    print(f'downloading and unzipping {fname}...', end=' ')
    r = urlopen(url)
    zipf = ZipFile(BytesIO(r.read()))
    zipf.extractall(path=save_dir)
    print(f'completed')
  else:
    print(f'{fname} already downloaded')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

sdsd

In [2]:
n_tokens = 400000 + 1 # glove vocabulary size + UNK + PAD

def get_glove(emb_size=100):
  """
    Download and load glove embeddings. 
    Parameters:
      size: embedding size (50/100/200/300-dimensional vectors).    
    Returns keras Embedding layer.  
  """
  if emb_size not in (50, 100, 200, 300):
    raise ValueError(f'wrong size parameter: {emb_size}')
  
  download_and_unzip('http://nlp.stanford.edu/data/glove.6B.zip', save_dir='glove')
  vocabulary = dict()
  embedding_matrix = np.ones((n_tokens, emb_size))

  with open(f'glove/glove.6B.{emb_size}d.txt') as f:
    for i, line in enumerate(f):
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embedding_matrix[i] = coefs
        vocabulary[word] = i
  
  # add embedding for and padding
  embedding_matrix[n_tokens - 1] = 0
  vocabulary['<PAD>'] = n_tokens - 1

  return vocabulary, embedding_matrix

In [3]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels
    def __getitem__(self, idx):
        return self.inputs[idx], self.labels[idx]
    def __len__(self):
        return self.inputs.shape[0]

In [30]:
classes = ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 
        'NNP', 'NNPS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 
        'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '<PAD>'] # serve una classe per il padding vero?
class2idx = {c: i for i, c in enumerate(classes)}


def add_oov(start_voc, oov_voc, embedding_matrix, sentences):
  print(type(oov_voc), type(start_voc))
  oovs = oov_voc - set(start_voc.keys())
  start_voc_size, emb_size = embedding_matrix.shape
  oov_embeddings = np.zeros((start_voc_size + len(oovs), emb_size))
  oov_embeddings[:start_voc_size] = embedding_matrix
  new_voc = dict(start_voc)
  for i, oov in enumerate(oovs):
      context_words = []
      for s in filter(lambda s: oov in s, sentences):
          context_words += [new_voc[w] for w in s 
                            if w in new_voc and w not in (oov, '<PAD>')]
      oov_embeddings[i] = np.mean(oov_embeddings[context_words])
      new_voc[oov] = start_voc_size + i
  return new_voc, oov_embeddings
    
def load_data(start, end, start_voc, embedding_matrix, number_token=True,
              drop_punctuation=True, split_docs=True, ret_counts=False):
  """
    Downloads dataset and preprocess data.
    Params:
      start: 
      end: 
      start_voc:
      classes:
      drop_punctuation:
      split_docs:
      ret_counts:
    Returns 
  """
  # download dataset
  download_and_unzip('https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip')
  
  inputs, labels = [], []
  vocabulary = set()
  counts = defaultdict(int)
  max_seq_len = 0
  
  # build dataset
  for doc in range(start, end+1):
    with open(f'dependency_treebank/wsj_{doc:04d}.dp') as f:
      
      input_seq, label_seq = [], []
      
      for line in f:
        if line.strip(): # check for empty lines
          word, label, _ = line.split('\t')
          word = word.lower()
          if '\/' in word:
            word = word.replace('\/', '-')
          if not drop_punctuation or label.isalpha(): # eventually drop punctuation
            vocabulary.add(word)
            input_seq.append(word)
            label_seq.append(label)
            counts[word] += 1
        elif split_docs: # sentence over, add to input if splitting documents
          max_seq_len = max(max_seq_len, len(input_seq))
          inputs.append(input_seq)
          labels.append(label_seq)
          input_seq, label_seq = [], []

      max_seq_len = max(max_seq_len, len(input_seq))
      inputs.append(input_seq)
      labels.append(label_seq)
  
  for i_seq, l_seq in zip(inputs, labels):
    i_seq += ['<PAD>'] * (max_seq_len - len(i_seq))
    l_seq += ['<PAD>'] * (max_seq_len - len(l_seq))
    
  vocabulary, embedding_matrix = add_oov(start_voc, vocabulary, embedding_matrix, inputs)

  inputs = torch.as_tensor([[vocabulary[word] for word in sequence] for sequence in inputs])
  labels = torch.as_tensor([[class2idx[label] for label in sequence] for sequence in labels])

  if ret_counts:
    return inputs, labels, vocabulary, embedding_matrix, counts
  else:
    return inputs, labels, vocabulary, embedding_matrix

In [26]:
class POSTagger(torch.nn.Module):

  def __init__(self, embedding_matrix, type, rec_size=1, units=None, hid_size=50):
    """
      A recurrent network performing multiclass classification (POS tagging).
      Params:
        type: 
        embedding_matrix: 
        rec_size: 
        units: 
        hid_size: 
    """
    super().__init__()

    emb_size = embedding_matrix.shape[1]
    self.emb_layer = nn.Embedding.from_pretrained(torch.as_tensor(embedding_matrix))

    if type == 'lstm':
      rec_module = nn.LSTM
    elif type == 'gru':
      rec_module = nn.GRU
    else:
      raise ValueError(f'wrong type {type}, either lstm or gru')
    self.rec_modules = rec_module(input_size=emb_size, hidden_size=hid_size, bidirectional=True, batch_first=True, num_layers=rec_size)

    fc_params = [2 * hid_size] + ([units, 37] if units is not None else [37])
    self.fc_modules = nn.Sequential(
      OrderedDict([(f'fc_{i}', nn.Linear(in_shape, out_shape)) 
      for i, (in_shape, out_shape) in enumerate(zip(fc_params[:-1], fc_params[1:]))]))
      
    self.logsoftmax = nn.LogSoftmax(dim=1)

  def __call__(self, x):
    vecs = self.emb_layer(x).float()
    rec_out, _ = self.rec_modules(vecs)
    fc_out = self.fc_modules(rec_out)
    return self.logsoftmax(fc_out)

In [27]:
def train_one_epoch(model, optimizer, loss_fn, data_loader, device):
    model.train()
    log_dict = {'train/loss': []}

    for inputs, targets in data_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        logprobs = model(inputs).transpose(1, 2)
        loss = loss_fn(logprobs, targets)
        loss_value = loss.item()

        if not math.isfinite(loss_value):
            print(f"Loss is {loss_value}, stopping training")
            exit(1)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        log_dict['train/loss'].append(loss_value)

    return log_dict

def evaluate(model, loss_fn, data_loader, device, metric='accuracy'):
    model.eval()
    batch_losses = []
    batch_metrics = []

    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            logprobs = model(inputs).transpose(1, 2)
            loss_value = loss_fn(logprobs, targets).item()
            preds = torch.argmax(logprobs, 1)

            if metric == 'accuracy':
                metric_value = (targets == preds).sum() / (data_loader.batch_size * targets.shape[1])
            elif metric == 'f1':
                pass
            else:
                raise ValueError(f'wrong metric {metric}, either accuracy or f1')

            batch_losses.append(loss_value)
            batch_metrics.append(metric_value.item())

    log_dict = {'valid/loss': np.mean(batch_losses), 
               f'valid/{metric}': np.mean(batch_metrics)}
    return log_dict


In [34]:
def train():
    cfg_dict = {
        'epochs': cfg.EPOCHS, 'batch_size': cfg.BATCH_SIZE, 
        'model': cfg.TYPE, 'rec_size': cfg.REC_SIZE, 'units': cfg.UNITS, 'hid_size': cfg.HID_SIZE,
        'optim': cfg.OPTIM, 'lr': cfg.LR, 'alpha': cfg.ALPHA, 'betas': cfg.BETAS, 'momentum': cfg.MOMENTUM, 'weight_decay': cfg.WEIGHT_DECAY
    }

    print('CONFIG PARAMETERS:')
    pprint(cfg_dict)

    wandb.login(key='bb91b382cc121df7e109ec0ad0275f1accc4c2f4')
    run = wandb.init(project="assignment-one", entity="nlpetroni", reinit=True, config=cfg_dict)
    wandb.define_metric("train_step")
    wandb.define_metric("epoch")
    wandb.define_metric('train/loss', step_metric="train_step", summary="min")
    wandb.define_metric("valid/loss", step_metric="epoch", summary="min")
    wandb.define_metric("valid/accuracy", step_metric="epoch", summary="max")

    glove_voc, embedding_matrix = get_glove()
    train_set, train_labels, train_voc, embedding_matrix = load_data(1, 100, glove_voc, embedding_matrix)
    valid_set, valid_labels, valid_voc, embedding_matrix = load_data(101, 150, glove_voc, embedding_matrix)


    train_dl = torch.utils.data.DataLoader(Dataset(train_set, train_labels), batch_size=cfg.BATCH_SIZE)
    valid_dl = torch.utils.data.DataLoader(Dataset(valid_set, valid_labels), batch_size=cfg.BATCH_SIZE)
    
    model = POSTagger(embedding_matrix, type=cfg.TYPE, rec_size=cfg.REC_SIZE, units=cfg.UNITS, hid_size=cfg.HID_SIZE).to(device)
    wandb.watch(model, log_graph=True)
    print(summary(model))

    params = [p for p in model.parameters() if p.requires_grad]
    if cfg.OPTIM == 'rmsprop':
        optimizer = torch.optim.RMSprop(params, lr=cfg.LR, alpha=cfg.ALPHA, momentum=cfg.MOMENTUM, weight_decay=cfg.WEIGHT_DECAY)
    elif cfg.OPTIM == 'adam':
        optimizer = torch.optim.Adam(params, lr=cfg.LR, betas=cfg.BETAS, weight_decay=cfg.WEIGHT_DECAY)
    else:
        raise ValueError(f'wrong optim {cfg.OPTIM}, either rmsprop or adam')
    loss = nn.NLLLoss()

    train_step = 0
    print('STARTING TRAINING')
    
    for epoch in range(cfg.EPOCHS):
        log_dict = train_one_epoch(model, optimizer, loss, train_dl, device)
        log_dict.update(evaluate(model, loss, valid_dl, device))
        for batch_loss in log_dict['train/loss']:
            wandb.log({'train_step': train_step, 'epoch': epoch, 'train/loss': batch_loss})
            train_step += 1
        wandb.log({'epoch': epoch, 'valid/loss': log_dict['valid/loss'], 'valid/accuracy': log_dict['valid/accuracy']})
        if (epoch % 50) == 0:
            print(f'[{epoch:03d}/{cfg.EPOCHS:03d}] train loss: {np.mean(log_dict["train/loss"]):.2f}, valid_loss:{log_dict["valid/loss"]:.2f}')
    run.finish()

    return model


In [35]:
models = []
for lr in (0.01, 0.001):
    cfg.LR = lr
    for batch_size in (64, 128):
        cfg.BATCH_SIZE = batch_size
        for optim in ('rmsprop', 'adam'):
            cfg.OPTIM = optim
            for (rec_size, units) in ( (1, None), (2, None), (1, 16), (1, 32)):
                cfg.REC_SIZE = rec_size
                cfg.UNITS = units
                model = train()
                models.append(model)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


CONFIG PARAMETERS:
{'alpha': 0.99,
 'batch_size': 64,
 'betas': (0.9, 0.999),
 'epochs': 250,
 'hid_size': 20,
 'lr': 0.01,
 'model': 'lstm',
 'momentum': 0.5,
 'optim': 'rmsprop',
 'rec_size': 1,
 'units': None,
 'weight_decay': 0}


wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


glove already downloaded
dependency_treebank already downloaded
<class 'set'> <class 'dict'>
dependency_treebank already downloaded
<class 'set'> <class 'dict'>


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


Layer (type:depth-idx)                   Param #
POSTagger                                --
├─Embedding: 1-1                         (40,054,500)
├─LSTM: 1-2                              19,520
├─Sequential: 1-3                        --
│    └─Linear: 2-1                       1,517
├─LogSoftmax: 1-4                        --
Total params: 40,075,537
Trainable params: 21,037
Non-trainable params: 40,054,500
STARTING TRAINING
[000/250] train loss: 4.85, valid_loss:3.59


KeyboardInterrupt: 

In [ ]:
test_set, test_labels, test_voc = load_data(151, 199, glove_voc, classes)
test_dl = torch.utils.data.DataLoader(Dataset(test_set, test_labels))   